### 自定义初始化参数

In [ ]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        init.constant_(param, val=0)
        print(name, param.data)

调用了 pytorch torch.nn 内的 init 模块将该参数以均值 mean 标准差 std 进行初始化。含normal_、constant_等

In [ ]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()
        return tensor

自定义初始化函数，在此过程中不记录梯度信息

### 自定义层（含模型参数）

In [ ]:
class MyDense(nn.Module):
    def __init__(self):
        super(MyDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))
    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
    
net = MyDense()
print(net)

注意此处添加参数的方式为使用 `ParameterList` 构造的参数列表，可以使用如 append 等方法

In [ ]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                    'linear1': nn.Parameter(torch.randn(4, 4)),
                    'linear2': nn.Parameter(torch.randn(4, 1))
                    })
        self.params.update({'linear3': nn.Parameter(torch.randn(4,
        2))}) # 􀷛􀥀
    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])
    
net = MyDictDense()
print(net)

x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))

而 `ParameterDict` 则可以按字典方式进行处理，并且根据传入的键值来进行不同的前向传播